### <span style="color: #1f77b4;">Problem Statement</span>

Understanding the problem statement is the first and foremost step. This would help you give an intuition of what you will face ahead of time. Let us see the problem statement.

<span style="color: #1f77b4; font-weight: bold;">Nairobi Housing Finance Company</span> offers all types of housing loans. They are present in all urban, semi-urban, and rural locations. Customers apply for a house loan once the company verifies their loan eligibility. The organization wants to <span style="font-style: italic;">automate the loan eligibility process (in real time)</span> based on the information provided by the consumer when filling out the online application form. These characteristics include <span style="font-weight: bold;">Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History</span>, and others. To automate this process, they created a problem to identify <span style="font-style: italic; color: #1f77b4;">client segments that are eligible for loan amounts</span> so that they may directly target these customers.

### <span style="color: #1f77b4;">Problem Statement</span>

It is a <span style="color: #2ca02c; font-weight: bold;">classification problem</span> where we have to predict whether a loan would be approved or not. In these kinds of problems, we have to predict <span style="font-style: italic;">discrete values</span> based on a given set of independent variables (s).

- <span style="font-weight: bold;">Binary Classification</span>: In this context, binary classification refers to predicting either of the two given classes. For example, classifying loan applications as either <span style="font-style: italic;">approved</span> or <span style="font-style: italic;">rejected</span> based on customer data.

- <span style="font-weight: bold;">MultiClass Classification</span>: Here, multiclass classification involves categorizing loan applications into multiple classes. For instance, you might classify loan applications into categories like <span style="font-style: italic;">approved</span>, <span style="font-style: italic;">rejected</span>, and <span style="font-style: italic;">under review</span> based on customer attributes.

<span style="font-style: italic;">Loan prediction</span> is a very common real-life problem that each retail bank faces at least once in its lifetime. If done correctly, it can save a lot of man-hours at the end of a retail bank.

---

![Loan prediction](images/image_1.png)

### <span style="color: #1f77b4;">Hypothesis Generation</span>

After looking at the problem statement, we will now move into hypothesis generation. It is the process of listing out all the possible factors that can affect the outcome.

Below are some of the factors which I think can affect the Loan Approval (dependent variable for this loan prediction problem):

- <span style="color: black; font-weight: bold;">Salary</span>: Applicants with high income should have more chances of getting approval.
- <span style="color: black; font-weight: bold;">Previous History</span>: Applicants who have paid their historical debts have more chances of getting approval.
- <span style="color: black; font-weight: bold;">Loan Amount</span>: Less the amount higher the chances of getting approval.
- <span style="color: black; font-weight: bold;">Loan Term</span>: Less the time period has higher chances of approval.
- <span style="color: black; font-weight: bold;">EMI</span>: Lesser the amount to be paid monthly, the higher the chances of getting approval.

---